In [39]:
import numpy as np
import pandas as pd

## Inputs

In [40]:
desired_mon = "Rookidee"
biome = "forest"
preset = ""
nearby_blocks = ""
can_see_sky = "true"

In [41]:
file_path = 'cobb.csv'
rawData = pd.read_csv(file_path)

## Preprocessing

In [42]:
rawData = rawData.dropna(how='all')
rawData = rawData.replace(np.nan, '')
data = rawData.applymap(lambda x: x.lower() if type(x) == str else x)

## Filtering

In [43]:
def filter_column(row, value, column):
    if ',' in row[column]:
        return value in row[column]
    else:
        return row[column] == value

In [44]:
if biome != '':
    filtered_basic_data = data[data.apply(filter_column, value=biome, column='Biome', axis=1) | (data['Biome'].str.contains('overworld'))]
else:
    filtered_basic_data = data[(data['Biome'].str.contains('overworld')) | data['Biome'] == '']
    
if preset != '':
    filtered_basic_data = filtered_basic_data[data.apply(filter_column, value=preset, column='Preset', axis=1) | (filtered_basic_data['Preset'] == '')]
else:
    filtered_basic_data = filtered_basic_data[filtered_basic_data['Preset'] == '']


if nearby_blocks != '':
    filtered_basic_data = filtered_basic_data[(filtered_basic_data['Requirements'].str.contains(nearby_blocks)) | (filtered_basic_data['Requirements'] == '')]
else:
    filtered_basic_data = filtered_basic_data[filtered_basic_data['Requirements'] == '']

filtered_basic_data = filtered_basic_data[filtered_basic_data['canseeSky'] == can_see_sky] # TODO AND ANY

In [45]:
alist = filtered_basic_data['Pokemon'].tolist()
alist.sort()
print(alist)

['ariados', 'braviary', 'corviknight', 'corvisquire', 'gallade', 'heracross', 'ironvaliant', 'kirlia', 'noctowl', 'pikipek', 'ralts', 'roaringmoon', 'rookidee', 'rufflet', 'spinarak', 'sudowoodo', 'toucannon', 'trumbeak']


## Calculating rarities

In [46]:
def calculate_rarities(bucket, data):
    bucket_data = data[data['Bucket'] == bucket]
    
    groupings = {
        ('sun', 'clear'): [],
        ('sun', 'rain'): [],
        ('sun', 'storm'): [],
        ('night', 'clear'): [],
        ('night', 'rain'): [],
        ('night', 'storm'): [],
    }
    groupings_with_any = {}
    
    # Add mons to their respective groups
    for index, row in bucket_data.iterrows():
        time = row['Time']
        weather = row['Weather']
        
        new_data = (row['Pokemon'], row['Weight'])
        
        key = (time, weather)
        target_dict = groupings_with_any if time == 'any' or weather == 'any' else groupings
        
        if key in target_dict:
            target_dict[key].append(new_data)
        else:
            target_dict[key] = [new_data]

    # Add combinations with 'any' to the other groupings
    for combination, pokemon in groupings_with_any.items():
        curr_time, curr_weather = combination
        
        # any and any
        if curr_time == curr_weather:
            for key in groupings.keys():
                groupings[key].extend(pokemon)  
        
        # any and whatever
        elif curr_time == 'any':
            for key in groupings.keys():
                extended_time, extended_weather = key
                if curr_time == extended_time:
                    groupings[key].extend(pokemon)
                    
        # whatever and any
        else:
            for key in groupings.keys():
                extended_time, extended_weather = key
                if curr_weather == extended_weather:
                    groupings[key].extend(pokemon)
            
    
    percentages = {}
                
    for combination, pokemon in groupings.items():
        max_weight = sum(weight for mon, weight in pokemon)
        
        percentages[combination] = [(mon, weight, f"{weight / max_weight * 100:.2f}%") for mon, weight in pokemon]
    
    return percentages
            

## Results

In [47]:
bucket_options = ['common', 'uncommon', 'rare', 'ultra-rare']

for bucket in bucket_options:
    bucket_rarities = calculate_rarities(bucket, filtered_basic_data)
    print(f"Bucket: {bucket.capitalize()}")
    
    for combo, pokemons in bucket_rarities.items():
        curr_time, curr_weather = combo
        
        pokemon_str = ', '.join([f'{mon.capitalize()} - {chances}' for mon, weight, chances in pokemons])
        result = f"Time: {curr_time.capitalize()}, Weather: {curr_weather.capitalize()} -> {pokemon_str}"
        print(f'{result}')
    
    print("\n")

Bucket: Common
Time: Sun, Weather: Clear -> Rookidee - 90.00%, Corvisquire - 9.00%, Corviknight - 1.00%
Time: Sun, Weather: Rain -> Rookidee - 90.00%, Corvisquire - 9.00%, Corviknight - 1.00%
Time: Sun, Weather: Storm -> Rookidee - 90.00%, Corvisquire - 9.00%, Corviknight - 1.00%
Time: Night, Weather: Clear -> Noctowl - 5.88%, Spinarak - 31.76%, Ariados - 3.53%, Rookidee - 52.94%, Corvisquire - 5.29%, Corviknight - 0.59%
Time: Night, Weather: Rain -> Rookidee - 90.00%, Corvisquire - 9.00%, Corviknight - 1.00%
Time: Night, Weather: Storm -> Rookidee - 90.00%, Corvisquire - 9.00%, Corviknight - 1.00%


Bucket: Uncommon
Time: Sun, Weather: Clear -> Ralts - 44.98%, Kirlia - 4.50%, Gallade - 0.45%, Pikipek - 24.99%, Trumbeak - 24.99%, Ironvaliant - 0.09%
Time: Sun, Weather: Rain -> Ralts - 44.98%, Kirlia - 4.50%, Gallade - 0.45%, Pikipek - 24.99%, Trumbeak - 24.99%, Ironvaliant - 0.09%
Time: Sun, Weather: Storm -> Ralts - 44.98%, Kirlia - 4.50%, Gallade - 0.45%, Pikipek - 24.99%, Trumbeak -